<a id="item41"></a>

## PART 2 .VGG16 Model

In [38]:
!wget "https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip"


Saved under concrete_data_week4.zip


In [ ]:
!unzip -q concrete_data_week4.zip

After you unzip the data, you fill find the data has already been divided into a train, validation, and test sets.

<a id="item42"></a>


1. Import libraries, modules, and packages you will need. Make sure to import the *preprocess_input* function from <code>keras.applications.vgg16</code>.
2. Use a batch size of 100 images for both training and validation.
3. Construct an ImageDataGenerator for the training set and another one for the validation set. VGG16 was originally trained on 224 × 224 images, so make sure to address that when defining the ImageDataGenerator instances.
4. Create a sequential model using Keras. Add VGG16 model to it and dense layer.
5. Compile the mode using the adam optimizer and the categorical_crossentropy loss function.
6. Fit the model on the augmented data using the ImageDataGenerators.

In [1]:
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense

Leveraging the VGG16 model to build our classifier, so let's download it as well.

In [7]:
from keras.applications import VGG16
from keras.applications.vgg16 import preprocess_input

Define Global Constants

We are obviously dealing with two classes, so num_classes is 2.
The VGG16 model was built and trained using images of size (224 x 224). Therefore, we will have to resize our images from (227 x 227) to (224 x 224).
We will training and validating the model using batches of 100 images.

In [18]:
num_classes = 2
image_resize = 224
batch_size_training = 100
batch_size_validation = 100

Construct ImageDataGenerator Instances

In [ ]:
data_generator= ImageDataGenerator(preprocessing_function=preprocess_input,)

Next, we will use the flow_from_directory method to get the training images

In [22]:
train_generator= data_generator.flow_from_directory('/concrete_data_week4/train',target_size=(image_resize,image_resize),
                                                   batch_size=batch_size_training,class_mode='categorical')

Found 30001 images belonging to 2 classes.


Validation generator

In [ ]:
valid_generator= data_generator.flow_from_directory('/concrete_data_week4/valid',target_size=(image_resize,image_resize),
                                                   batch_size=batch_size_validation,class_mode='categorical')

Build and Compile the model

In [24]:
model= Sequential()
model.add(VGG16(include_top=False,pooling='avg',weights='imagenet'))



58889256/58889256 [==============================] - 13s 0us/step


In [25]:
model.add(Dense(num_classes, activation='softmax'))

In [26]:
model.layers

In [27]:
model.layers[0].layers

In [28]:
model.layers[0].trainable = False

In [29]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 512)               14714688  
                                                                 
 dense (Dense)               (None, 2)                 1026      
                                                                 
Total params: 14715714 (56.14 MB)
Trainable params: 1026 (4.01 KB)
Non-trainable params: 14714688 (56.13 MB)
_________________________________________________________________


In [33]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [34]:
steps_per_epoch_training = len(train_generator)
steps_per_epoch_validation = len(train_generator)
num_epochs = 2

In [36]:
fit_history = model.fit_generator(
    train_generator,
    steps_per_epoch=steps_per_epoch_training,
    epochs = num_epochs,
    validation_data=valid_generator,
    validation_steps=steps_per_epoch_validation,
    verbose=1,
)

C:\Users\aiswa\AppData\Local\Temp\ipykernel_33248\3578781097.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  fit_history = model.fit_generator(


Epoch 1/2


301/301 [==============================] - 14866s 49s/step - loss: 0.0709 - accuracy: 0.9783 - val_loss: 0.0231 - val_accuracy: 0.9945
Epoch 2/2
301/301 [==============================] - 20652s 69s/step - loss: 0.0169 - accuracy: 0.9957


In [37]:
model.save('/classifier_vgg16_model.h5')

D:\AnacondaInstallation\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


<a id="item43"></a>

## Part 2 Evaluate Deep Learning Models using Test Data

1. Load your saved model that was built using the ResNet50 model. 
2. Construct an ImageDataGenerator for the test set. For this ImageDataGenerator instance, you only need to pass the directory of the test images, target size, and the **shuffle** parameter and set it to False.
3. Use the **evaluate_generator** method to evaluate your models on the test data, by passing the above ImageDataGenerator as an argument. You can learn more about **evaluate_generator** [here](https://keras.io/models/sequential/).
4. Print the performance of the classifier using the VGG16 pre-trained model.
5. Print the performance of the classifier using the ResNet pre-trained model.


Use the following cells to evaluate your models.

In [41]:
from keras.models import load_model
model_ResNet= load_model("/classifier_resnet_model.h5")
model_ResNet.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 dense (Dense)               (None, 2)                 4098      
                                                                 
Total params: 23591810 (90.00 MB)
Trainable params: 4098 (16.01 KB)
Non-trainable params: 23587712 (89.98 MB)
_________________________________________________________________


In [42]:
from keras.models import load_model
model_VGG16= load_model("/classifier_vgg16_model.h5")
model_VGG16.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 512)               14714688  
                                                                 
 dense (Dense)               (None, 2)                 1026      
                                                                 
Total params: 14715714 (56.14 MB)
Trainable params: 1026 (4.01 KB)
Non-trainable params: 14714688 (56.13 MB)
_________________________________________________________________


###### Construct an ImageDataGenerator for Test Set

In [43]:
test_generator= data_generator.flow_from_directory('/concrete_data_week4/test',target_size=(image_resize,image_resize),
                                                   shuffle=False)

Found 500 images belonging to 2 classes.


Use the evaluate_generator method to evaluate your models on the test data, by passing the above ImageDataGenerator as an argument.
 [evaluate_generator](https://keras.io/models/sequential/)

In [46]:
resnet50_accuracy = model_ResNet.evaluate_generator(test_generator)
print(resnet50_accuracy)

C:\Users\aiswa\AppData\Local\Temp\ipykernel_33248\2416416246.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  resnet50_accuracy = model_ResNet.evaluate_generator(test_generator)


[0.009541066363453865, 0.9980000257492065]


In [48]:
vgg16_accuracy = model_VGG16.evaluate_generator(test_generator)
print(vgg16_accuracy)

C:\Users\aiswa\AppData\Local\Temp\ipykernel_33248\1473358092.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  vgg16_accuracy = model_VGG16.evaluate_generator(test_generator)


[0.011607191525399685, 0.9959999918937683]


<a id="item44"></a>

## Part 3 Predict whether the images in the test data are images of cracked concrete or not

1. Use the **predict_generator** method to predict the class of the images in the test data, by passing the test data ImageDataGenerator instance defined in the previous part as an argument. You can learn more about the **predict_generator** method [here](https://keras.io/models/sequential/).


Use the following cells to make your predictions.

In [67]:

Resnet_predictor=model_ResNet.predict_generator(test_generator)


C:\Users\aiswa\AppData\Local\Temp\ipykernel_33248\99088406.py:1: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  Resnet_predictor=model_ResNet.predict_generator(test_generator)


In [68]:
vgg16_predictor = model_VGG16.predict_generator(test_generator)

C:\Users\aiswa\AppData\Local\Temp\ipykernel_33248\2493975669.py:1: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  vgg16_predictor = model_VGG16.predict_generator(test_generator)


In [71]:
import numpy as np
predictResnet= [np.argmax(predictions) for predictions in Resnet_predictor]
predictVGG16=[np.argmax(predictions) for predictions in vgg16_predictor]
    

In [76]:
pred_Resnet=[]
for i in predictResnet:
  if i==0:
    pred_Resnet.append('negative')
  else:
    pred_Resnet.append('positive')
print(pred_Resnet[:5])

['negative', 'negative', 'negative', 'negative', 'negative']


In [77]:
pred_VGG16=[]
for i in predictVGG16:
  if i==0:
    pred_VGG16.append('negative')
  else:
    pred_VGG16.append('positive')
print(pred_VGG16[:5])

['negative', 'negative', 'negative', 'negative', 'negative']
